In [5]:
from src.preprocessing import split_on_heuristics
from src.dictionary_utils import extract_wordforms, extend_dictionary, build_trie
from src.ngram_utils import load_unigrams, load_bigrams
from src.segmentation import segment_text_fast
from src.utils import get_space_indices
import kagglehub
import pandas as pd

In [ ]:
!unzip data/dictionary.txt.zip -d data
!unzip data/bigrams.zip -d data
!unzip data/unigrams.zip  -d data

## Load data

In [44]:
texts = pd.read_csv(
    "data/dataset_1937770_3.txt",
    header=None,
    names=['index', 'text_no_spaces'],
    index_col=False,
    engine='python',
)
texts = texts.drop(columns="index")
texts = texts.iloc[1:]

/tmp/ipython-input-3722828685.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  texts = pd.read_csv(


In [56]:
path = kagglehub.dataset_download("antonoof/avito-data")
print("Path to dataset files:", path)
df_train = pd.read_parquet('/kaggle/input/avito-data/data/train_part_0001.snappy.parquet')

Using Colab cache for faster access to the 'avito-data' dataset.
Path to dataset files: /kaggle/input/avito-data


## Build dataset

In [57]:
dictionary = extract_wordforms("data/dictionary.txt")
top_avito = extend_dictionary(df_train, top_k=1000)
dictionary = sorted(set(top_avito) | set(dictionary))
trie = build_trie(dictionary)

100%|██████████| 500000/500000 [00:01<00:00, 313916.62it/s]


## Add ngrams

In [58]:
unigrams = load_unigrams("data/unigrams")
bigrams = load_bigrams("data/bigrams")

## Get predictions

In [59]:
texts['bigrams'] = texts['text_no_spaces'].apply(lambda x: segment_text_fast(x, trie, bigrams, unigrams))

In [60]:
texts['predicted_positions'] = texts['bigrams'].apply(get_space_indices)

In [61]:
texts['predicted_positions']

,predicted_positions
1,"[5, 10, 12]"
2,"[3, 6, 7]"
3,"[4, 12, 13, 20, 21]"
4,"[5, 10, 18]"
5,"[5, 10]"
...,...
1001,"[1, 3]"
1002,"[6, 7, 10, 12, 16, 18]"
1003,"[3, 11, 19]"
1004,"[6, 8]"


In [63]:
texts['id'] = texts.index
res = texts[['id', "predicted_positions"]]

In [67]:
res.to_csv("baseline.csv",index=False)